In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from ast import literal_eval
from sklearn.model_selection import train_test_split
import sys
sys.path.append("../")

from cfmining.algorithms import MAPOFCEM
from cfmining.criteria import PercentileCalculator, PercentileCriterion, PercentileChangesCriterion, NonDomCriterion
from cfmining.predictors import GeneralClassifier_Shap
from cfmining.action_set import ActionSet

import dice_ml


%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
VAL_RATIO = 1/7
TEST_RATIO = 0.3
SEED = 0

df = pd.read_csv("../data/german.csv")
X = df.drop("GoodCustomer", axis=1)
Y = df["GoodCustomer"]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=TEST_RATIO, random_state=SEED, shuffle=True)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=VAL_RATIO, random_state=SEED, shuffle=True)

In [8]:
action_set = ActionSet(X = X_train)
not_mutable_features = ['Age', 'OwnsHouse', 'isMale', 'JobClassIsSkilled', 'Single', 'ForeignWorker', 'RentsHouse']
mutable_features = [feat for feat in X_train.columns if feat not in not_mutable_features]
continuous_features = ['Age', 'LoanDuration', 'LoanAmount', 'LoanRateAsPercentOfIncome','YearsAtCurrentHome']
for feat in action_set:
    if feat.name in not_mutable_features:
        feat.mutable = False
    if not feat.name in not_mutable_features:
        feat.mutable = True

    feat.step_direction = 0
    feat.update_grid()
percCalc = PercentileCalculator(action_set = action_set)

In [3]:
dice = pd.read_csv("../results/german/dice_best_sol_0.csv")

In [11]:
dice["individual"].apply(lambda x : x.replace("  ", ","))

0     [[,,0,, 1,,42,,60 10366,, 2,, 4,, 1,, 1,, 1,, ...
1     [[, 0,,1, 43, 24 7393,,1,,4,,1,,2,,0,,0,,0,,0,...
2     [[,0, 0,42,12 409, 3, 3, 2, 1, 0, 1, 1, 1, 1, ...
3     [[, 0,,1, 24, 48 6560,,3,,2,,1,,1,,0,,1,,0,,1,...
4     [[, 0,,1, 29, 36 9034,,4,,1,,1,,1,,1,,1,,0,,1,...
                            ...                        
73    [[,0, 1,30,12 707, 4, 2, 2, 1, 0, 0, 0, 0, 0, ...
74    [[, 0,,1, 38, 15 1308,,4,,4,,2,,1,,0,,1,,0,,0,...
75    [[, 0,,0, 20, 11 1577,,4,,1,,1,,1,,0,,1,,0,,1,...
76    [[, 0,,1, 23, 45 1845,,4,,4,,1,,1,,1,,0,,0,,0,...
77    [[, 0,,0, 25, 12 1295,,3,,1,,1,,1,,0,,1,,0,,0,...
Name: individual, Length: 78, dtype: object